In [1]:
# ------------------ SETUP ------------------
!pip install transformers datasets faiss-cpu sentence-transformers gradio pandas PyMuPDF --quiet
!pip install gradio
!pip install PyMuPDF
!pip install faiss-cpu
!pip install sentence-transformers
# ------------------ IMPORTS ------------------
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from sentence_transformers import SentenceTransformer, util
import gradio as gr
import pandas as pd
import fitz  # PyMuPDF
import os
import faiss
import numpy as np

# ------------------ STEP 1A: LOAD QA PAIRS FROM CSV ------------------
csv_path = "/content/FINAL_MERGE_IPC_Final_all.csv"
df = pd.read_csv(csv_path)
df = df.dropna(subset=["Question", "Answer"])
questions = df["Question"].tolist()
answers = df["Answer"].tolist()

# ------------------ STEP 1B: EXTRACT & CHUNK TEXT FROM PDF ------------------
pdf_path = "/content/merged.pdf"
pdf_text = ""
if os.path.exists(pdf_path):
    doc = fitz.open(pdf_path)
    for page in doc:
        pdf_text += page.get_text()
    doc.close()
    chunk_size = 500
    chunk_overlap = 100
    pdf_chunks = [pdf_text[i:i+chunk_size] for i in range(0, len(pdf_text), chunk_size - chunk_overlap)]
else:
    pdf_chunks = []

# ------------------ STEP 2: EMBEDDINGS ------------------
embedder = SentenceTransformer('all-MiniLM-L6-v2')
question_embeddings = embedder.encode(questions, convert_to_numpy=True)
pdf_embeddings = embedder.encode(pdf_chunks, convert_to_numpy=True) if pdf_chunks else []

# Build FAISS index for questions
qa_dim = question_embeddings.shape[1]
qa_index = faiss.IndexFlatL2(qa_dim)
qa_index.add(np.array(question_embeddings))

# Build FAISS index for PDF (optional)
if pdf_chunks:
    pdf_dim = pdf_embeddings.shape[1]
    pdf_index = faiss.IndexFlatL2(pdf_dim)
    pdf_index.add(np.array(pdf_embeddings))

# ------------------ STEP 3: HYBRID QA RETRIEVAL ------------------
def answer_question(user_question):
    query_embedding = embedder.encode(user_question, convert_to_numpy=True)

    # 1. Search QA pairs
    D, I = qa_index.search(np.array([query_embedding]), k=3)
    response = ""
    for i, idx in enumerate(I[0]):
        q = questions[idx]
        a = answers[idx]
        response += f"**Relevant Q{i+1}:** {q}\n\n**Answer:** {a}\n\n---\n"

    # 2. Fallback to PDF chunks if score too high (poor match)
    if D[0][0] > 1.0 and pdf_chunks:
        D_pdf, I_pdf = pdf_index.search(np.array([query_embedding]), k=1)
        best_chunk = pdf_chunks[I_pdf[0][0]]
        response += f"\n**📄 Extracted from Constitution/PDF:**\n\n{best_chunk[:500]}..."

    return response.strip()

# ------------------ STEP 4: GRADIO UI ------------------
interface = gr.Interface(
    fn=answer_question,
    inputs=gr.Textbox(lines=2, placeholder="Ask your legal question related to IPC, Constitution, etc..."),
    outputs=gr.Markdown(),
    title="📘 Indian Law Legal QA Assistant (FAISS-Powered)",
    description="Ask legal questions based on Indian law. Fast semantic search over QA and PDF sources using FAISS."
)

interface.launch(share=True)


Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
                      ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 95, in resolve
    result = self._result = resolver.resolve(
                            ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_vendor/resolvelib/resolvers.py", line 546, in resolve
    state = resolution.resolve(requirements, max_rounds=max_rounds)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://648620bc1810217421.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
